In [1]:
import os
import pandas as pd
import geopandas as gpd

In [9]:


# Folder path containing the CSV files
folder_path = r'D:\HLandscape\11.PDF\航测范围'

# Get the list of all CSV files in the folder
all_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]

# Read and concatenate the CSV files
df_list = []
for file in all_files:
    df = pd.read_csv(file)
    # Delete the first column
    df = df.iloc[:, 1:]
    df_list.append(df)

# Concatenate the dataframes
combined_df = pd.concat(df_list, ignore_index=True)

# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(
    combined_df, geometry=gpd.points_from_xy(combined_df['经度'], combined_df['纬度']))

# Set the coordinate reference system (CRS) to WGS84
gdf.set_crs(epsg=4326, inplace=True)

# Export the GeoDataFrame as a GeoJSON file
gdf.to_file("combined_coordinates.geojson", driver="GeoJSON")


In [10]:
gdf

,地点,经度,纬度,Table Name,geometry
0,后店村,116.251900,40.547000,丰宁县-01,POINT (116.25190 40.54700)
1,富贵山村,116.221700,41.218900,丰宁县-02,POINT (116.22170 41.21890)
2,九宫号村,116.175900,41.032500,丰宁县-03,POINT (116.17590 41.03250)
3,高栅子村,116.211000,40.595900,丰宁县-04,POINT (116.21100 40.59590)
4,杨木栅子村,116.223900,40.581200,丰宁县-05,POINT (116.22390 40.58120)
...,...,...,...,...,...
259,木兰林场北沟分场黄家东沟,117.481875,41.836140,木兰围场林场-41,POINT (117.48188 41.83614)
260,木兰林场八英庄分场光顶山林路旁,117.948071,42.020240,木兰围场林场-42,POINT (117.94807 42.02024)
261,木兰林场八英庄分场贾家梁,117.864605,42.066896,木兰围场林场-43,POINT (117.86460 42.06690)
262,木兰林场八英庄分场下窝铺,117.869296,42.054027,木兰围场林场-44,POINT (117.86930 42.05403)


In [23]:
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster

# 读取GeoJSON文件
geojson_file = "./combined_coordinates.geojson"
gdf = gpd.read_file(geojson_file)

# 检查数据
print(gdf.head())

# 创建一个Folium地图
m = folium.Map(location=[gdf.geometry.y.mean(), gdf.geometry.x.mean()], zoom_start=12)

# 添加Google Satellite图层
folium.TileLayer(
    tiles='http://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
    attr='Google',
    name='Google Satellite',
    overlay=True,
    control=True
).add_to(m)

# 添加一个标记集群
marker_cluster = MarkerCluster().add_to(m)

# 添加每个点到地图上，并使用“地点”作为标签
for idx, row in gdf.iterrows():
    folium.Marker(location=[row.geometry.y, row.geometry.x],
                  popup=row['地点'],
                  icon=folium.Icon(color='blue')).add_to(marker_cluster)

# 添加图层控制
folium.LayerControl().add_to(m)

# 保存地图
m.save('interactive_map.html')

# 显示地图
m


      地点        经度       纬度 Table Name                    geometry
0    后店村  116.2519  40.5470     丰宁县-01  POINT (116.25190 40.54700)
1   富贵山村  116.2217  41.2189     丰宁县-02  POINT (116.22170 41.21890)
2   九宫号村  116.1759  41.0325     丰宁县-03  POINT (116.17590 41.03250)
3   高栅子村  116.2110  40.5959     丰宁县-04  POINT (116.21100 40.59590)
4  杨木栅子村  116.2239  40.5812     丰宁县-05  POINT (116.22390 40.58120)
